In [5]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import explained_variance_score, r2_score
from sklearn.model_selection import cross_validate
import tensorflow as tf
from sklearn.metrics import mean_squared_error as MSE
from pathlib import Path
from sklearn.metrics import classification_report
import xgboost as xg 
from xgboost import XGBClassifier
import numpy as np
import pandas as pd 



In [6]:
pip install csv-to-sqlite

Note: you may need to restart the kernel to use updated packages.


In [7]:
import csv_to_sqlite

In [8]:
options = csv_to_sqlite.CsvOptions(typing_style="full", encoding="UTF-8")
#change path to "Resources/insurance.csv" for jupyter notebook
input_files = ['Resources/insurance.csv']
#change path to "Resources/insurance.sqlite" for jupyter notebook
csv_to_sqlite.write_csv(input_files, "Resources/insurance.sqlite", options)


Written 1338 rows into 1 tables in 0.018 seconds


1338

In [11]:
import sqlalchemy
from sqlalchemy import create_engine, text, inspect, func
import warnings
engine = create_engine("sqlite:///Resources/insurance.sqlite")

#gives warning but easier to get the dataframe from querying
warnings.filterwarnings('ignore')
conn = engine.raw_connection()
temp_f = pd.read_sql_query(sql=f"SELECT * FROM insurance", con=conn)

In [ ]:

#  Import and read the insurance.csv.

insurance_df = pd.read_csv('Resources/insurance.csv')
insurance_df.head()

In [2]:
# Cleaning DataFrame - drop extra index column
insurance_df = insurance_df.drop(columns = ['index'])
insurance_df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


SKIP THIS# Cleaning DataFrame - cleaning charges column
insurance_df['charges'] = insurance_df['charges'].astype('float64')
insurance_df['charges'] = insurance_df['charges'].map("{:,.2f}".format)
insurance_df

In [ ]:
insurance_df.dtypes

In [4]:
# Define features set
X = insurance_df.copy()
X.drop("charges", axis=1, inplace=True)
X.head()

,age,sex,bmi,children,smoker,region
0,19,female,27.900,0,yes,southwest
1,18,male,33.770,1,no,southeast
2,28,male,33.000,3,no,southeast
3,33,male,22.705,0,no,northwest
4,32,male,28.880,0,no,northwest


In [5]:
# Define target vector
y = insurance_df["charges"].values.reshape(-1, 1)
y[:5]

array([[16884.924  ],
       [ 1725.5523 ],
       [ 4449.462  ],
       [21984.47061],
       [ 3866.8552 ]])

In [6]:
# Encode the categorical variables using get_dummies
X = pd.get_dummies(X)

In [7]:
X[:5]

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,1,0,0,1,0,0,0,1
1,18,33.770,1,0,1,1,0,0,0,1,0
2,28,33.000,3,0,1,1,0,0,0,1,0
3,33,22.705,0,0,1,1,0,0,1,0,0
4,32,28.880,0,0,1,1,0,0,1,0,0


In [8]:
# Splitting 
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                      test_size = 0.3, random_state = 123) 

# XGboost

In [9]:

# Instantiation 


xgb_r  = GridSearchCV(
            estimator=xg.XGBRegressor(),
            param_grid={"learning_rate": (0.05, 0.10, 0.15),
                        "max_depth": [ 3, 4, 5, 6, 8, None],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "colsample_bytree":[ 0.3, 0.4, 0.8]},
             scoring='neg_mean_absolute_error', verbose=3, n_jobs=-1)

grid_result = MultiOutputRegressor(xgb_r).fit(train_X, train_y)
best_params = grid_result.best_params_

Fitting 5 folds for each of 648 candidates, totalling 3240 fits
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1;, score=-3691.409 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3;, score=-3532.269 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3;, score=-3669.151 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=5;, score=-3671.138 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=5;, score=-3276.465 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=5;, score=-3403.370 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=5;, score=-3534.205 total time

[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3;, score=-3400.347 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=4, min_child_weight=5;, score=-3690.277 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=1;, score=-3875.801 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=5;, score=-3519.661 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=5;, score=-3614.287 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=6, min_child_weight=3;, score=-3711.462 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=6, min_child_weight=3;, score=-3885.064 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_

[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1;, score=-3677.262 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=4, min_child_weight=5;, score=-3541.395 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=1;, score=-3470.340 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=5;, score=-3830.173 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=5, min_child_weight=5;, score=-3445.419 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=6, min_child_weight=5;, score=-3937.501 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=6, min_child_weight=5;, score=-3485.611 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_

[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.05, max_depth=6, min_child_weight=7;, score=-3689.497 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.05, max_depth=6, min_child_weight=7;, score=-3855.288 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=6, min_child_weight=5;, score=-3116.115 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=6, min_child_weight=5;, score=-3184.988 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=6, min_child_weight=5;, score=-3292.173 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=6, min_child_weight=7;, score=-3422.214 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=6, min_child_weight=7;, score=-3140.915 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=

[CV 3/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=4, min_child_weight=1;, score=-2916.769 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=4, min_child_weight=1;, score=-2760.611 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=4, min_child_weight=1;, score=-2901.326 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=4, min_child_weight=3;, score=-3107.880 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=4, min_child_weight=3;, score=-2983.369 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=4, min_child_weight=3;, score=-2859.024 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=4, min_child_weight=3;, score=-2883.929 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.2, learning_

[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=None, min_child_weight=3;, score=-3218.977 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=None, min_child_weight=3;, score=-3250.232 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=None, min_child_weight=5;, score=-3444.654 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=None, min_child_weight=5;, score=-3164.680 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=None, min_child_weight=5;, score=-3116.115 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=None, min_child_weight=5;, score=-3184.988 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=None, min_child_weight=5;, score=-3292.173 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=

[CV 2/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=8, min_child_weight=7;, score=-3219.341 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=8, min_child_weight=7;, score=-3152.955 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=8, min_child_weight=7;, score=-3196.965 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=8, min_child_weight=7;, score=-3288.546 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=None, min_child_weight=1;, score=-3543.620 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=None, min_child_weight=1;, score=-3248.960 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.1, max_depth=None, min_child_weight=1;, score=-3254.686 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learnin

[CV 1/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=7;, score=-3240.024 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=7;, score=-3102.036 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=7;, score=-2991.005 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=7;, score=-3013.973 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=7;, score=-2995.757 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=6, min_child_weight=1;, score=-3470.582 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=6, min_child_weight=1;, score=-3281.607 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.2, learning_

[CV 3/5] END colsample_bytree=0.4, gamma=0.1, learning_rate=0.15, max_depth=None, min_child_weight=7;, score=-3181.707 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.1, learning_rate=0.15, max_depth=None, min_child_weight=7;, score=-3023.741 total time=   0.0s
[CV 5/5] END colsample_bytree=0.4, gamma=0.1, learning_rate=0.15, max_depth=None, min_child_weight=7;, score=-2980.484 total time=   0.0s
[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=3, min_child_weight=1;, score=-3127.276 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=3, min_child_weight=1;, score=-2929.035 total time=   0.0s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=3, min_child_weight=1;, score=-3041.222 total time=   0.0s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=3, min_child_weight=1;, score=-2969.801 total time=   0.0s
[CV 5/5] END colsample_bytree=0.4, gamma=0.2, 

AttributeError: 'MultiOutputRegressor' object has no attribute 'best_params_'

In [16]:
xgb_r  = GridSearchCV(
            estimator=xg.XGBRegressor(),
            param_grid={"learning_rate": (0.05, 0.10, 0.15),
                        "n_estimators" : (100, 200, 300, 500),
                        "max_depth": [ 3, 4, 5, 6, 8, None],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "colsample_bytree":[ 0.3, 0.4, 0.8]},
             scoring='neg_mean_absolute_error', verbose=3, n_jobs=-1)

In [17]:
xgb_r.fit(train_X, train_y)

Fitting 5 folds for each of 2592 candidates, totalling 12960 fits
[CV 4/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=5, min_child_weight=3;, score=-2993.858 total time=   0.0s
[CV 5/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=5, min_child_weight=3;, score=-2944.717 total time=   0.0s
[CV 1/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=5, min_child_weight=5;, score=-3135.402 total time=   0.0s
[CV 2/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=5, min_child_weight=5;, score=-3054.596 total time=   0.0s
[CV 3/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=5, min_child_weight=5;, score=-3079.685 total time=   0.0s
[CV 4/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=5, min_child_weight=5;, score=-2883.884 total time=   0.0s
[CV 5/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.15, max_depth=5, min_child_weight=5;, score=-2909.471 total ti

[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=300;, score=-2918.557 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=500;, score=-2862.069 total time=   0.1s
[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=500;, score=-2738.069 total time=   0.1s
[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3, n_estimators=100;, score=-3400.347 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3, n_estimators=100;, score=-3532.269 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min_child_weight=3, n_estimators=100;, score=-3669.151 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.05, max_depth=3, min

[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=200;, score=-2998.225 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=300;, score=-3399.665 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=300;, score=-3142.951 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=300;, score=-3316.839 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=300;, score=-3014.377 total time=   0.1s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=300;, score=-3067.912 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.1, max_depth=5, min_child_

[CV 2/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=5, min_child_weight=5, n_estimators=300;, score=-3068.133 total time=   0.1s
[CV 3/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=5, min_child_weight=5, n_estimators=300;, score=-3150.209 total time=   0.0s
[CV 4/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=5, min_child_weight=5, n_estimators=300;, score=-2995.706 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=5, min_child_weight=5, n_estimators=300;, score=-3105.644 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=5, min_child_weight=5, n_estimators=500;, score=-3357.129 total time=   0.1s
[CV 3/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=6, min_child_weight=3, n_estimators=300;, score=-3226.830 total time=   0.1s
[CV 4/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.05, max_depth=6, min

[CV 5/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.1, max_depth=8, min_child_weight=7, n_estimators=200;, score=-3244.965 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.1, max_depth=8, min_child_weight=7, n_estimators=300;, score=-3658.496 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.1, max_depth=8, min_child_weight=7, n_estimators=300;, score=-3429.998 total time=   0.1s
[CV 5/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.1, max_depth=None, min_child_weight=3, n_estimators=100;, score=-3250.232 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.1, max_depth=None, min_child_weight=3, n_estimators=200;, score=-3575.812 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.1, max_depth=None, min_child_weight=3, n_estimators=200;, score=-3257.782 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.1, learning_rate=0.1, max_depth=None

...

[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=7, n_estimators=500;, score=-3112.311 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=7, n_estimators=500;, score=-3014.698 total time=   0.1s
[CV 3/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=7, n_estimators=500;, score=-2965.055 total time=   0.1s
[CV 4/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=7, n_estimators=500;, score=-2808.050 total time=   0.1s
[CV 5/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=7, n_estimators=500;, score=-2867.896 total time=   0.1s
[CV 1/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=6, min_child_weight=1, n_estimators=100;, score=-3647.686 total time=   0.0s
[CV 2/5] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=6, min

[CV 3/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=500;, score=-3565.990 total time=   0.1s
[CV 4/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=500;, score=-3265.806 total time=   0.1s
[CV 5/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=500;, score=-3435.432 total time=   0.1s
[CV 1/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=7, n_estimators=100;, score=-2988.480 total time=   0.0s
[CV 2/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=7, n_estimators=100;, score=-3019.358 total time=   0.0s
[CV 3/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=7, n_estimators=100;, score=-2928.828 total time=   0.0s
[CV 4/5] END colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_

[CV 1/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=5, min_child_weight=5, n_estimators=500;, score=-3615.392 total time=   0.1s
[CV 5/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=6, min_child_weight=3, n_estimators=100;, score=-2914.042 total time=   0.0s
[CV 1/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=6, min_child_weight=3, n_estimators=200;, score=-3355.583 total time=   0.1s
[CV 2/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=6, min_child_weight=3, n_estimators=200;, score=-3320.379 total time=   0.0s
[CV 3/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=6, min_child_weight=3, n_estimators=200;, score=-3250.895 total time=   0.1s
[CV 4/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=6, min_child_weight=3, n_estimators=200;, score=-3109.379 total time=   0.0s
[CV 5/5] END colsample_bytree=0.8, gamma=0.0, learning_rate=0.1, max_depth=6, min_child_

[CV 2/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=300;, score=-2771.128 total time=   0.1s
[CV 3/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=300;, score=-2752.785 total time=   0.0s
[CV 4/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=300;, score=-2620.514 total time=   0.0s
[CV 5/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=300;, score=-2630.314 total time=   0.0s
[CV 1/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_depth=3, min_child_weight=5, n_estimators=500;, score=-2883.763 total time=   0.1s
[CV 1/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_depth=4, min_child_weight=5, n_estimators=100;, score=-2840.883 total time=   0.0s
[CV 2/5] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_depth=4, min

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bi...
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.3, 0.4, 0.8],
                         'gamma': [0.0, 0.1, 0.2],
                         'learning_rate': (0.05, 0.1, 0.15),
                         'max_depth': [3, 4, 5, 6, 8, None],
                         'min_child_weight': [1, 3, 5, 7],
                         'n_estimators': (100, 200, 300, 500)},
             scoring='neg_mean_absolute_error', verbose=3)

In [18]:
xgb_r.best_score_

-2619.405075574804

In [39]:
best_param = xgb_r.best_params_
best_param

{'colsample_bytree': 0.8,
 'gamma': 0.0,
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_weight': 1,
 'n_estimators': 100}

In [33]:
# now we take the best possible params and build the model for training
xgboost_model = xgboost.XGBRegressor(n_estimators = best_param['n_estimators'],
                                     colsample_bytree=best_param['colsample_bytree'],
                                     learning_rate = best_param['learning_rate'],
                                     max_depth = best_param['max_depth'],
                                     min_child_weight=best_param['min_child_weight'])

In [34]:
# Fitting the model 
xgboost_model.fit(train_X, train_y) 

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [35]:
pred = xgb_r.predict(test_X)

In [36]:
# RMSE Computation 
rmse = np.sqrt(MSE(test_y, pred)) 
print("RMSE : % f" %(rmse)) 

RMSE :  4109.559429


In [41]:
xval_score = cross_validate(xgboost_model,
                            train_X, train_y,
                            scoring = ['r2', 'explained_variance'], verbose=3,
                            return_train_score=True, n_jobs=-1)
xval_score


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


{'fit_time': array([0.02609491, 0.02525902, 0.0187881 , 0.01402903, 0.01418304]),
 'score_time': array([0.00412107, 0.00279617, 0.00164413, 0.00124598, 0.0012219 ]),
 'test_r2': array([0.85788655, 0.78939986, 0.8490695 , 0.86501179, 0.85169603]),
 'train_r2': array([0.89208709, 0.90111757, 0.89534063, 0.89181671, 0.89389724]),
 'test_explained_variance': array([0.85791833, 0.78965494, 0.84951208, 0.86502817, 0.85170725]),
 'train_explained_variance': array([0.89208742, 0.9011179 , 0.89534077, 0.89181695, 0.89389748])}

[CV] END ...................... r2: (train=0.894, test=0.852) total time=   0.0s
[CV] END  explained_variance: (train=0.892, test=0.858) r2: (train=0.892, test=0.858) total time=   0.0s
[CV] END  explained_variance: (train=0.901, test=0.790) r2: (train=0.901, test=0.789) total time=   0.0s
[CV] END  explained_variance: (train=0.895, test=0.850) r2: (train=0.895, test=0.849) total time=   0.0s
[CV] END  explained_variance: (train=0.892, test=0.865) r2: (train=0.892, test=0.865) total time=   0.0s
[CV] END  explained_variance: (train=0.894, test=0.852) r2: (train=0.894, test=0.852) total time=   0.0s
